In [ ]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
import re
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
df = pd.read_csv('/content/Laptop_Train_v2.csv')

# Filter out rows with 'conflict' sentiment
df = df[df['polarity'] != 'conflict']

df

,id,Sentence,Aspect Term,polarity,from,to
0,2339,I charge it at night and skip taking the cord ...,cord,neutral,41,45
1,2339,I charge it at night and skip taking the cord ...,battery life,positive,74,86
2,1316,The tech guy then said the service center does...,service center,negative,27,41
3,1316,The tech guy then said the service center does...,"""sales"" team",negative,109,121
4,1316,The tech guy then said the service center does...,tech guy,neutral,4,12
...,...,...,...,...,...,...
2352,2272,We also use Paralles so we can run virtual mac...,Windows 7 Home Premium,neutral,80,102
2353,2272,We also use Paralles so we can run virtual mac...,Windows Server Enterprise 2003,neutral,104,134
2354,2272,We also use Paralles so we can run virtual mac...,Windows Server 2008 Enterprise,neutral,140,170
2356,848,"How Toshiba handles the repair seems to vary, ...",repair,positive,130,136


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

# Load the ABSA model and tokenizer
model_name = "yangheng/deberta-v3-base-absa-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

for aspect in ['cord', 'battery life']:
   print(aspect, classifier('I charge it at night and skip taking the cord with me because of the good battery life.',  text_pair=aspect))


/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


cord [{'label': 'Neutral', 'score': 0.983368456363678}]
battery life [{'label': 'Positive', 'score': 0.9977699518203735}]


In [ ]:
for aspect in ['system', 'software']:
   print(aspect, classifier('When I finally had everything running with all my software installed I plugged in my droid to recharge and the system crashed.',  text_pair=aspect))

system [{'label': 'Negative', 'score': 0.9949234127998352}]
software [{'label': 'Positive', 'score': 0.747990608215332}]


In [ ]:
for aspect in ['performance', 'memory']:
   print(aspect, classifier('I have been impressed with the battery life and the performance for such a small amount of memory.',  text_pair=aspect))

performance [{'label': 'Positive', 'score': 0.9963219165802002}]
memory [{'label': 'Negative', 'score': 0.9939292669296265}]


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Extract values from DataFrame columns
texts = df['Sentence'].tolist()
aspects = df['Aspect Term'].tolist()
sentiments = df['polarity'].tolist()

# Convert sentiment labels to numerical values
label_encoder = LabelEncoder()
numerical_sentiments = label_encoder.fit_transform(sentiments)

# Split the data into train and test sets
train_texts, test_texts, train_aspects, test_aspects, train_sentiments, test_sentiments = train_test_split(
    texts, aspects, numerical_sentiments, test_size=0.2, random_state=42
)

# Load pre-trained model and tokenizer
model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Ensure the model's output layer matches the number of classes
num_classes = len(set(numerical_sentiments))
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_classes)

# Tokenize input data for both train and test sets
tokenized_train = tokenizer(train_texts, train_aspects, padding=True, truncation=True, return_tensors="pt")
tokenized_test = tokenizer(test_texts, test_aspects, padding=True, truncation=True, return_tensors="pt")

# Create TensorDatasets for train and test sets
train_dataset = TensorDataset(tokenized_train["input_ids"], tokenized_train["attention_mask"], torch.tensor(train_sentiments))
test_dataset = TensorDataset(tokenized_test["input_ids"], tokenized_test["attention_mask"], torch.tensor(test_sentiments))

# Fine-tune the model on the training set
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# Training loop (simplified example)
num_epochs = 4
for epoch in range(num_epochs):
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Evaluate the model on the test set
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1).cpu().numpy()
        all_preds.extend(predictions)
        all_labels.extend(labels.cpu().numpy())

# Convert numerical sentiments back to original labels
predicted_sentiments = label_encoder.inverse_transform(all_preds)
true_sentiments = label_encoder.inverse_transform(all_labels)

# Print classification report
print(classification_report(true_sentiments, predicted_sentiments))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


              precision    recall  f1-score   support

    negative       0.87      0.81      0.84       181
     neutral       0.64      0.61      0.62       114
    positive       0.80      0.88      0.84       168

    accuracy                           0.79       463
   macro avg       0.77      0.77      0.77       463
weighted avg       0.79      0.79      0.78       463



In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import LabelEncoder
import pandas as pd


# Extract values from DataFrame columns
texts = df['Sentence'].tolist()
aspects = df['Aspect Term'].tolist()
sentiments = df['polarity'].tolist()

# Use a label encoder to convert string labels to numerical values
label_encoder = LabelEncoder()
numerical_sentiments = label_encoder.fit_transform(sentiments)

# Load the ABSA model and tokenizer
model_name = "yangheng/deberta-v3-base-absa-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSequenceClassification.from_pretrained(model_name,)

# Ensure the model's output layer matches the number of classes
num_classes = len(set(numerical_sentiments))
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_classes)

# Tokenize input data
tokenized_inputs = tokenizer(texts, aspects, padding=True, truncation=True, return_tensors="pt")

# Create TensorDataset
input_ids = tokenized_inputs["input_ids"]
attention_mask = tokenized_inputs["attention_mask"]
labels = torch.tensor(numerical_sentiments)  # Use numerical labels

dataset = TensorDataset(input_ids, attention_mask, labels)

# Split the data into train and test sets
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

# Fine-tune the model on the training set
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# Training loop (simplified example)
num_epochs = 1
for epoch in range(num_epochs):
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Evaluate the model on the test set
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1).cpu().numpy()
        all_preds.extend(predictions)
        all_labels.extend(labels.cpu().numpy())

# Print classification report
print(classification_report(all_labels, all_preds))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.


              precision    recall  f1-score   support

           0       0.98      0.97      0.98       181
           1       0.93      0.92      0.93       114
           2       0.95      0.96      0.96       168

    accuracy                           0.96       463
   macro avg       0.95      0.95      0.95       463
weighted avg       0.96      0.96      0.96       463



In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import LabelEncoder
import pandas as pd


# Extract values from DataFrame columns
texts = df['Sentence'].tolist()
aspects = df['Aspect Term'].tolist()
sentiments = df['polarity'].tolist()

# Use a label encoder to convert string labels to numerical values
label_encoder = LabelEncoder()
numerical_sentiments = label_encoder.fit_transform(sentiments)

# Load the ABSA model and tokenizer
model_name = "yangheng/deberta-v3-base-absa-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Tokenize input data
tokenized_inputs = tokenizer(texts, aspects, padding=True, truncation=True, return_tensors="pt")

# Create TensorDataset
input_ids = tokenized_inputs["input_ids"]
attention_mask = tokenized_inputs["attention_mask"]
labels = torch.tensor(numerical_sentiments)  # Use numerical labels

dataset = TensorDataset(input_ids, attention_mask, labels)

# Split the data into train and test sets
_, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

# Evaluate the model on the test set
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1).cpu().numpy()
        all_preds.extend(predictions)
        all_labels.extend(labels.cpu().numpy())

# Convert numerical sentiments back to original labels
predicted_sentiments = label_encoder.inverse_transform(all_preds)
true_sentiments = label_encoder.inverse_transform(all_labels)

# Print classification report
print(classification_report(true_sentiments, predicted_sentiments))


/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


              precision    recall  f1-score   support

    negative       0.97      0.96      0.97       181
     neutral       0.91      0.96      0.93       114
    positive       0.96      0.94      0.95       168

    accuracy                           0.95       463
   macro avg       0.95      0.95      0.95       463
weighted avg       0.95      0.95      0.95       463



In [ ]:
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch
        outputs = loaded_model(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1).cpu().numpy()
        all_preds.extend(predictions)
        all_labels.extend(labels.cpu().numpy())

# Convert numerical sentiments back to original labels
predicted_sentiments = label_encoder.inverse_transform(all_preds)
true_sentiments = label_encoder.inverse_transform(all_labels)

# Print classification report
print(classification_report(true_sentiments, predicted_sentiments))


              precision    recall  f1-score   support

    negative       0.88      0.70      0.78       181
     neutral       0.51      0.70      0.59       114
    positive       0.79      0.77      0.78       168

    accuracy                           0.73       463
   macro avg       0.73      0.72      0.72       463
weighted avg       0.76      0.73      0.73       463

